# Extract all possible occurrences of “Audit & Inspection Rights” clause

## Load and Chunk 


In [77]:

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredWordDocumentLoader


def load_and_chunk_document(docx_path):
    """Loads document and intelligently chunks while preserving section integrity."""
    loader = UnstructuredWordDocumentLoader(docx_path, mode="elements")
    data = loader.load()

    # Extract text and clean it
    text = "\n".join([chunk.page_content.strip() for chunk in data if chunk.page_content.strip()])

    #  chunking
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,  # Large enough to capture a full clause
        chunk_overlap=200,  # Overlap to prevent split clauses
        separators=["\n\n", "\n", ". ", " "],  # Prefer splitting at paragraphs or sentences
    )
    
    chunks = text_splitter.split_text(text)
    
    return chunks

# Load and chunk document
docx_path = "contract_interview.docx"
document_chunks = load_and_chunk_document(docx_path)





In [78]:
len(document_chunks)

196

In [79]:
# Print first few chunks 
for i, chunk in enumerate(document_chunks[:15]):
    print(f"\n🔹🔹🔹 Chunk {i+1} (Length: {len(chunk)} characters):\n{chunk}\n\n")
    print("-----------------")


🔹🔹🔹 Chunk 1 (Length: 464 characters):
Exhibit 10.2
CONFIDENTIAL
CERTAIN CONFIDENTIAL PORTIONS OF THIS EXHIBIT WERE OMITTED AND REPLACED WITH “[***]”. A COMPLETE VERSION OF THIS EXHIBIT HAS BEEN FILED SEPARATELY WITH THE SECRETARY OF THE SECURITIES AND EXCHANGE COMMISSION PURSUANT TO AN APPLICATION REQUESTING CONFIDENTIAL TREATMENT UNDER RULE 24b-2 OF THE SECURITIES EXCHANGE ACT OF 1934.
API COMMERCIAL SUPPLY AGREEMENT
by and between
.
and
.
Dated as of May 25, 2011
API COMM ERCIAL SUPPLY AGREEMENT


-----------------

🔹🔹🔹 Chunk 2 (Length: 818 characters):
API COMMERCIAL SUPPLY AGREEMENT
by and between
.
and
.
Dated as of May 25, 2011
API COMM ERCIAL SUPPLY AGREEMENT
THIS API COMMERCIAL SUPPLY AGREEMENT (this “Agreement”) is entered into and dated as of the 25th day of May, 2011 (the “Effective Date”) by and between  ., a corporation organized under the laws of Ireland and having its principal office at First Floor, Block 3, The Oval, Shelbourne Road, Ballsbridge, Dublin 4, Ireland (“”

## Import Requirements

In [80]:
import pandas as pd

from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor


from langchain_google_genai import ChatGoogleGenerativeAI



## Import ENVs

In [81]:
dotenv_path = 'D:\\Dhananjay\\Pro\\DCode\\MyAIPractice\\.env'

# import dotenv
import os
from dotenv import load_dotenv
# Load environment variables
load_dotenv(dotenv_path)

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
GEMINI_LLM_MODEL_NAME = os.getenv("GEMINI_LLM_MODEL_NAME")
GEMINI_EMBEDDING_MODEL_NAME = os.getenv("GEMINI_EMBEDDING_MODEL_NAME")


## Initialize LLM (Google Gemini)


In [94]:
GEMINI_LLM_MODEL_NAME = "gemini-2.5-pro-exp-03-25"

# Initialize LLM (Google Gemini)
gemini_2_5 = ChatGoogleGenerativeAI(
    google_api_key=GEMINI_API_KEY,
    model=GEMINI_LLM_MODEL_NAME,
    temprature = 0
)


gemini_2_5.invoke("who are you?")

AIMessage(content='I am a large language model, trained by Google.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-47adcd7e-bc90-42e3-912e-3e1b8d8d17ef-0', usage_metadata={'input_tokens': 5, 'output_tokens': 11, 'total_tokens': 16, 'input_token_details': {'cache_read': 0}})

In [82]:

# Initialize LLM (Google Gemini)
model = ChatGoogleGenerativeAI(
    google_api_key=GEMINI_API_KEY,
    model=GEMINI_LLM_MODEL_NAME,
    temprature = 0
)

In [83]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

def get_embedding_model():
    """
    Returns the configured embedding model for vector storage and retrieval.
    """
    google_embedding = GoogleGenerativeAIEmbeddings(
        model=GEMINI_EMBEDDING_MODEL_NAME,
        google_api_key=GEMINI_API_KEY
    )

    return google_embedding

In [84]:
 from langchain_core.documents import Document

# Convert text chunks to LangChain Document objects
documents = [Document(page_content=chunk) for chunk in document_chunks]




## Retriever

In [85]:
from langchain_community.vectorstores import Chroma


# Add to vectorDB
vectorstore = Chroma.from_documents(
    documents=documents,
    collection_name="extract-clause-chroma",
    embedding=get_embedding_model(),
    persist_directory="./chroma_langchain_db",  #  save data locally

)

retriever = vectorstore.as_retriever()

###  Create Retriever Tool

In [86]:
from langchain.tools.retriever import create_retriever_tool

# Create a retriever tool to extract "Audit & Inspection Rights" clauses
retriever_tool = create_retriever_tool(
    retriever,  
    "retrieve_audit_inspection_clauses",
    "Extract all occurrences of the 'Audit & Inspection Rights' clause from the document. "
)

# Define tools list
tools = [retriever_tool]

# Example usage (if integrated in an agent-based workflow)
result = retriever_tool.invoke("Extract all 'Audit & Inspection Rights' clauses in the document")
print(result)


9.2 Audit and Inspection Rights. During the Term of this Agreement and thereafter during any applicable records retention period(s) under Section 9.3, Approved Representatives shall have the right, upon a prior written consent of , not to be unreasonably withheld or delayed, to audit and inspect those portions of the Facility (or the facility of a Material Third Party Supplier or Subcontractor, as the case may be) used in, and those documents and records related to, the manufacture, generation, storage, testing, treatment, holding, transportation, distribution or other handling or receiving of the API and Third Party Materials. Such audits may be conducted [***] each [***]; provided, however, that  may conduct additional “for cause” audits during a [***] to the extent  supplies Nonconforming API or in the event of Product complaints or Adverse Events caused by Nonconforming API.  may redact from such deliveries to  any Third Party confidential information or  Confidential Information



# Agents

### Audit_Inspection_Extraction_Agent

In [88]:
Audit_Inspection_Extraction_Agent_Prompt = """Extract all possible occurrences of the "Audit & Inspection Rights" clause from the provided document.

Identify clauses that explicitly grant one party the right to:
- Audit or inspect the books, records, accounts, or financial documents of another party.
- Inspect or examine the premises, facilities, or operational sites of the other party.

The extracted clauses should contain a contractual obligation related to audits or inspections.

You have access to the `retrieve_audit_inspection_clauses` tool.

Return **only** the exact clause text as it appears in the contract. Do **not** generate any additional text, summaries, or explanations.

"""


In [89]:
Audit_Inspection_Extraction_Agent = create_react_agent(
    model=model,
    tools=tools,
    name="Audit Inspection Extraction Agent",
    prompt=Audit_Inspection_Extraction_Agent_Prompt
)


In [90]:
tool_messages = {
            "messages": [
                {
                    "role": "user",
                    "content": "Extract all possible occurrences of the `Audit & Inspection Rights` clause from the provided document."
                }
            ]
        }
    
    
tool_result = Audit_Inspection_Extraction_Agent.invoke(tool_messages)

In [91]:
for m in tool_result["messages"]:
    m.pretty_print()


================================ Human Message =================================

Extract all possible occurrences of the `Audit & Inspection Rights` clause from the provided document.
================================== Ai Message ==================================
Name: Audit Inspection Extraction Agent
Tool Calls:
  retrieve_audit_inspection_clauses (d3043354-e9e5-47a0-ab0c-03d41e9b664e)
 Call ID: d3043354-e9e5-47a0-ab0c-03d41e9b664e
  Args:
    query: Audit & Inspection Rights
================================= Tool Message =================================
Name: retrieve_audit_inspection_clauses

9.2 Audit and Inspection Rights. During the Term of this Agreement and thereafter during any applicable records retention period(s) under Section 9.3, Approved Representatives shall have the right, upon a prior written consent of , not to be unreasonably withheld or delayed, to audit and inspect those portions of the Facility (or the facility of a Material Third Party Supplier or Subcontrac

### KeyClauseConceptsAgent

In [42]:
Key_Clause_Concepts_Agent_Prompt = """Ensure the extracted clause strictly follows the scope of "Audit & Inspection Rights." 

Only validate clauses that explicitly grant one party the right to audit, inspect, or examine the books, records, accounts, or premises of the other party. 

Exclude any clauses related to government/regulatory audits, product inspections, or definitions of "auditor."

Return only the exact text from the contract document. Do not include any explanations, summaries, or extra text.
"""


In [43]:
Key_clause_concepts_agent = create_react_agent(
    model=model,
    tools=[],
    name="Key ClauseConcepts Agent",
    prompt=Key_Clause_Concepts_Agent_Prompt
)


### FeatureCheckAgent

In [44]:
Expected_Key_Clause_Feature_Agent_Prompt = """Ensure the clause contains at least one of the following rights:  

- The right of one party to audit the books and records of another party.  
- The right of one party to give or receive access to the books and records of another party.  
- The right to inspect or examine the premises, books, accounts, records, papers, or other specified items.  
- The obligation of one party to maintain or retain records, files, books, accounts, or papers for the purpose of audit or inspection.  
- The obligation of a party to follow certain procedures and rules with respect to conducting the audit.  

Reject clauses that do not meet this requirement.  

Return only the exact clause text from the contract document. Do not generate any additional content.


"""

In [45]:

Feature_Check_Agent = create_react_agent(
    model=model,
    tools=[],
    name="Feature Check Agent",
    prompt=Expected_Key_Clause_Feature_Agent_Prompt
)

### ExclusionFilterAgent

In [46]:
Exclude_Clause_Concepts_Agent_Prompt = """
Filter out any clauses that fall into the following exclusions:  

- Governmental or regulatory audits and inspections.  
- Inspection rights on the delivery of products, especially those related to acceptance testing.  
- Definitions of the term "auditor."  

Reject any clause that fits these exclusions.  

Return only the exact clause text from the contract document. Do not add any explanations or extra text.


"""

In [47]:
Exclusion_Filter_Agent = create_react_agent(
    model=model,
    tools=[],
    name="Exclusion Filter Agent",
    prompt=Exclude_Clause_Concepts_Agent_Prompt
)

### supervised workflow

In [48]:
Supervisor_Workflow_Prompt = """You are a team supervisor managing Audit Inspection Extraction Agent, Feature Check Agent, and Exclusion Filter Agent. 
        To extract all occurrences of "Audit & Inspection Rights" clauses, use Audit_Inspection_Extraction_Agent. 
        To verify if the extracted clauses align with the expected key clause features, use Feature_Check_Agent.
        To remove any extracted clauses that match the excluded concepts, use Exclusion_Filter_Agent

        Extract all occurrences of `Audit & Inspection Rights` clauses, verify if the extracted clauses align with the expected key clause features, and remove any extracted clauses that match the excluded concepts.

        Return only the exact clause text provided. Do not generate any summaries, explanations, or extra text.

        If no valid clause is found, return `"NO CLAUSE FOUND"`.  
        

        """

        # If no valid clause is found, return `"NO CLAUSE FOUND"`.  
# 

In [49]:

workflow = create_supervisor(
    [Audit_Inspection_Extraction_Agent, Feature_Check_Agent, Exclusion_Filter_Agent],  
    model=model,
    prompt=Supervisor_Workflow_Prompt
)


In [50]:
# Compile 
app = workflow.compile()



In [51]:
len(document_chunks)

17

In [52]:
sup_document_text_list = load_and_chunk_document(docx_path)

sup_result_list = []


for sup_doc in sup_document_text_list:

    su_messages = {
            "messages": [
                {
                    "role": "user",
                    "content": sup_doc
                }
            ]
        }
    
    
    sup_result = Audit_Inspection_Extraction_Agent.invoke(su_messages)
    
    sup_result_content = sup_result["messages"][-1].content
    sup_result_list.append(sup_result_content)
    print(sup_result_content)
    print("-------------------")

I am sorry, but based on the text you provided, there are no "Audit & Inspection Rights" clauses.
-------------------
5.7 Audit Rights.  shall have the right, but not the obligation, to audit the Facility, including the systems and standard operating procedures used to manufacture API for , to assess ’s compliance with cGMP and the requirements of this Agreement.  shall have the right, but not the obligation, to audit ’s Third Party Suppliers and Subcontractors to assess their compliance with cGMP and the requirements of this Agreement.  may exercise such audit rights upon reasonable advance notice to  and at reasonable times and intervals.  shall bear the cost of any such audits, unless a material breach of cGMP or this Agreement is discovered during any such audit, in which case  shall bear the reasonable cost of such audit.  shall have the right to be present during any audit of a Third Party Supplier or Subcontractor.
-------------------
I am sorry, but there are no "Audit & Inspec

In [53]:
# sup_document_text_list = load_and_chunk_document(docx_path)

sup_result_list_1 = []


for sup_doc in sup_document_text_list[8:]:

    su_messages = {
            "messages": [
                {
                    "role": "user",
                    "content": sup_doc
                }
            ]
        }
    
    
    sup_result_1 = Audit_Inspection_Extraction_Agent.invoke(su_messages)
    
    sup_result_content_1 = sup_result_1["messages"][-1].content
    sup_result_list_1.append(sup_result_content_1)
    print(sup_result_content_1)
    print("-------------------")

7.2 Establishment of cGMP Facility.
(b)  shall have the right, pursuant to the audit procedures in Section 9.2, to have its Approved Representatives undertake a quality assurance audit of ’s procedures and facilities for API production as soon as practicable after the date the Expansion is completed.
-------------------
I am sorry, but I could not find any clauses related to "Audit & Inspection Rights" in the provided document.
-------------------
Article IX
Information; Access; Audit Rights
-------------------
9.2 Audit and Inspection Rights. During the Term of this Agreement and thereafter during any applicable records retention period(s) under Section 9.3, Approved Representatives shall have the right, upon a prior written consent of , not to be unreasonably withheld or delayed, to audit and inspect those portions of the Facility (or the facility of a Material Third Party Supplier or Subcontractor, as the case may be) used in, and those documents and records related to, the manufact

In [39]:
# Function to save extracted clauses to an Excel file
def save_to_excel(extracted_text, output_path="Extracted_Audit_Clauses.xlsx"):
    df = pd.DataFrame({"Extracted Content": [extracted_text]})
    df.to_excel(output_path, index=False)
    print(f" Extracted clauses saved to: {output_path}")

    
# Function to extract and validate clauses
def process_contract(docx_path):
    document_text = load_and_chunk_document(docx_path)[12]
    
    
    messages = {
        "messages": [
            {
                "role": "user",
                "content": document_text
            }
        ]
    }
    
    
    validated_clauses = app.invoke(messages)

    return validated_clauses
    
    # Step 3: Save to Excel
    # save_to_excel(validated_clauses)





result = process_contract(docx_path)


# for m in result["messages"]:
#     m.pretty_print()

In [40]:
for m in result["messages"]:
    m.pretty_print()

================================ Human Message =================================

(d) Duties. In performing its duties under this Section 5.4,  shall perform the following tasks, consistent with the Quality Agreement:
(i) implement and operate an ICH complaint stability program;
(ii) notify ’s head of regulatory affairs, or his or her designee, promptly, but within not more than [***], if any batch of API fails any stability tests; and
(iii) report to ’s head of regulatory affairs, or his or her designee, promptly, but within not more than [***], any Nonconformity, significant atypical results, deviations or adverse trends exhibited during final release or stability testing.
(e) Manufacturing Process Review. At either Party’s reasonable request, the Parties shall promptly meet, in person or telephonically, for the purpose of reviewing such matters related to manufacturing of the API as may be specified by a Party, including discussing strategies for improving the API manufacturing proc

In [20]:
len(document_chunks)


34

# Manual Chaining fo Agents

## Start with the Audit_Inspection_Extraction_Agent

In [55]:
document_text_list = load_and_chunk_document(docx_path)

result_list_1 = []


for doc in document_text_list[:10]:

    messages = {
            "messages": [
                {
                    "role": "user",
                    "content": doc
                }
            ]
        }
    
    
    result = Audit_Inspection_Extraction_Agent.invoke(messages)
    
    result_content = result["messages"][-1].content
    result_list_1.append(result_content)
    print(result_content)
    print("-------------------")

NO CLAUSE FOUND
-------------------
NO CLAUSE FOUND
-------------------
NO CLAUSE FOUND
-------------------
NO CLAUSE FOUND
-------------------
NO CLAUSE FOUND
-------------------
NO CLAUSE FOUND
-------------------
NO CLAUSE FOUND
-------------------
NO CLAUSE FOUND
-------------------
Within [***] from the date of receipt of written notice of any API Price change,  may request  to provide its API Price adjustment records to an independent, mutually agreed upon, reputable certified public accounting firm, which will audit such records and certify whether the price adjustments notified by  are correct and in accordance with the methodology described in Schedule 3.1(e).
-------------------
NO CLAUSE FOUND
-------------------


In [56]:
# document_text_list = load_and_chunk_document(docx_path)

result_list_2 = []


for doc in document_text_list[10:20]:

    messages = {
            "messages": [
                {
                    "role": "user",
                    "content": doc
                }
            ]
        }
    
    
    result = Audit_Inspection_Extraction_Agent.invoke(messages)
    
    result_content = result["messages"][-1].content
    result_list_2.append(result_content)
    print(result_content)
    print("-------------------")

NO CLAUSE FOUND
-------------------
(i) During development and Validation of the Initial Manufacturing Process,  will accommodate in person technical meetings at the Facility and technical inspections as reasonably requested by . Without limiting the foregoing, during process development and in support of API process characterization and Validation activities,  will be permitted to conduct reviews of the Facility and the pertinent records maintained by , subject to restriction on access to all  Confidential Information, in connection with the conduct of manufacturing, storage and testing of API, all upon ’s request and with reasonable notice to permit  to support such technical reviews.
(iii) With the prior written consent of the other Party, a Party may engage in teleconferences, in person meetings, Facility reviews, quality assurance audits, records reviews and other activities under this Agreement through its (or its Affiliates’) employees or consultants with a bona fide need to kno

In [58]:
# document_text_list = load_and_chunk_document(docx_path)

result_list_3 = []


for doc in document_text_list[20:25]:

    messages = {
            "messages": [
                {
                    "role": "user",
                    "content": doc
                }
            ]
        }
    
    
    result = Audit_Inspection_Extraction_Agent.invoke(messages)
    
    result_content = result["messages"][-1].content
    result_list_3.append(result_content)
    print(result_content)
    print("-------------------")

NO CLAUSE FOUND
-------------------
NO CLAUSE FOUND
-------------------
9.2 Audit and Inspection Rights. During the Term of this Agreement and thereafter during any applicable records retention period(s) under Section 9.3, Approved Representatives shall have the right, upon a prior written consent of , not to be unreasonably withheld or delayed, to audit and inspect those portions of the Facility (or the facility of a Material Third Party Supplier or Subcontractor, as the case may be) used in, and those documents and records related to, the manufacture, generation, storage, testing, treatment, holding, transportation, distribution or other handling or receiving of the API and Third Party Materials. Such audits may be conducted [***] each [***]; provided, however, that  may conduct additional “for cause” audits during a [***] to the extent  supplies Nonconforming API or in the event of Product complaints or Adverse Events caused by Nonconforming API.  may redact from such deliveries to 

In [59]:
# document_text_list = load_and_chunk_document(docx_path)

result_list_4 = []


for doc in document_text_list[25:30]:

    messages = {
            "messages": [
                {
                    "role": "user",
                    "content": doc
                }
            ]
        }
    
    
    result = Audit_Inspection_Extraction_Agent.invoke(messages)
    
    result_content = result["messages"][-1].content
    result_list_4.append(result_content)
    print(result_content)
    print("-------------------")

NO CLAUSE FOUND
-------------------
NO CLAUSE FOUND
-------------------
NO CLAUSE FOUND
-------------------
NO CLAUSE FOUND
-------------------
NO CLAUSE FOUND
-------------------


In [60]:
# document_text_list = load_and_chunk_document(docx_path)

result_list_5 = []


for doc in document_text_list[30:]:

    messages = {
            "messages": [
                {
                    "role": "user",
                    "content": doc
                }
            ]
        }
    
    
    result = Audit_Inspection_Extraction_Agent.invoke(messages)
    
    result_content = result["messages"][-1].content
    result_list_5.append(result_content)
    print(result_content)
    print("-------------------")

NO CLAUSE FOUND
-------------------
NO CLAUSE FOUND
-------------------
NO CLAUSE FOUND
-------------------
NO CLAUSE FOUND
-------------------


# Get all possible occurrences of “Audit & Inspection Rights” clause

In [63]:
def merge_valid_clauses(*lists):
    """Merges multiple lists while removing 'NO CLAUSE FOUND' entries."""
    final_list = []
    
    for result_list in lists:
        for clause in result_list:
            if clause.strip().lower() != "no clause found":  # Ignore invalid clauses
                final_list.append(clause)

    return final_list



# Merge lists while filtering out "NO CLAUSE FOUND"
final_result_list = merge_valid_clauses(result_list_1, result_list_2, result_list_3, result_list_4, result_list_5)

#  final merged list
for cl in final_result_list:
    print(cl)
    print("----------")


Within [***] from the date of receipt of written notice of any API Price change,  may request  to provide its API Price adjustment records to an independent, mutually agreed upon, reputable certified public accounting firm, which will audit such records and certify whether the price adjustments notified by  are correct and in accordance with the methodology described in Schedule 3.1(e).
----------
(i) During development and Validation of the Initial Manufacturing Process,  will accommodate in person technical meetings at the Facility and technical inspections as reasonably requested by . Without limiting the foregoing, during process development and in support of API process characterization and Validation activities,  will be permitted to conduct reviews of the Facility and the pertinent records maintained by , subject to restriction on access to all  Confidential Information, in connection with the conduct of manufacturing, storage and testing of API, all upon ’s request and with rea

# Chain to Expected_key_clause_feature_agent

In [64]:

exp_result_list = []


for doc in final_result_list:

    messages = {
            "messages": [
                {
                    "role": "user",
                    "content": doc
                }
            ]
        }
    
    
    result = Expected_key_clause_feature_agent.invoke(messages)
    
    result_content = result["messages"][-1].content
    exp_result_list.append(result_content)
    print(result_content)
    print("-------------------")

Within [***] from the date of receipt of written notice of any API Price change, may request to provide its API Price adjustment records to an independent, mutually agreed upon, reputable certified public accounting firm, which will audit such records and certify whether the price adjustments notified by are correct and in accordance with the methodology described in Schedule 3.1(e).
-------------------
(i) During development and Validation of the Initial Manufacturing Process, will accommodate in person technical meetings at the Facility and technical inspections as reasonably requested by . Without limiting the foregoing, during process development and in support of API process characterization and Validation activities, will be permitted to conduct reviews of the Facility and the pertinent records maintained by , subject to restriction on access to all Confidential Information, in connection with the conduct of manufacturing, storage and testing of API, all upon ’s request and with 

# Chain to Exclude_clause_concepts_agent

In [65]:

exclude_result_list = []


for doc in exp_result_list:

    messages = {
            "messages": [
                {
                    "role": "user",
                    "content": doc
                }
            ]
        }
    
    
    result = Exclude_clause_concepts_agent.invoke(messages)
    
    result_content = result["messages"][-1].content
    exclude_result_list.append(result_content)
    print(result_content)
    print("-------------------")

Within [***] from the date of receipt of written notice of any API Price change, may request to provide its API Price adjustment records to an independent, mutually agreed upon, reputable certified public accounting firm, which will audit such records and certify whether the price adjustments notified by are correct and in accordance with the methodology described in Schedule 3.1(e).
-------------------
(i) During development and Validation of the Initial Manufacturing Process, will accommodate in person technical meetings at the Facility and technical inspections as reasonably requested by . Without limiting the foregoing, during process development and in support of API process characterization and Validation activities, will be permitted to conduct reviews of the Facility and the pertinent records maintained by , subject to restriction on access to all Confidential Information, in connection with the conduct of manufacturing, storage and testing of API, all upon ’s request and with 

# save file to excel 

In [70]:
import pandas as pd

def save_list_to_excel(data_list, output_file="output.xlsx"):
    """Converts a list to a Pandas DataFrame and saves it as an Excel file."""
    df = pd.DataFrame({"Extracted Clauses": data_list})  # Convert list to DataFrame
    df.to_excel(output_file, index=False)  # Save to Excel
    print(f" Excel file saved: {output_file}")



# Save to Excel
save_list_to_excel(exclude_result_list, "Extracted_Audit_Clauses.xlsx")


 Excel file saved: Extracted_Audit_Clauses.xlsx


In [ ]:

exclude_result_list_1 = []


for doc in final_result_list:

    messages = {
            "messages": [
                {
                    "role": "user",
                    "content": doc
                }
            ]
        }
    
    
    result = Exclude_clause_concepts_agent.invoke(messages)
    
    result_content = result["messages"][-1].content
    exclude_result_list_1.append(result_content)
    print(result_content)
    print("-------------------")

In [288]:


messages_2 = {
        "messages": [
            {
                "role": "user",
                "content": result_2
            }
        ]
    }

In [281]:
result_3 = Expected_key_clause_feature_agent.invoke(messages_2)

In [282]:
result_4 = result_3["messages"][-1].content
result_4

'quality assurance audits, records reviews and other activities under this Agreement through its (or its Affiliates’) employees or consultants with a bona fide need to know, but only to the extent necessary for the Party to exercise its rights and discharge its obligations under this Agreement,'

In [283]:
messages_3 = {
        "messages": [
            {
                "role": "user",
                "content": result_4
            }
        ]
    }

result_5 = Exclude_clause_concepts_agent.invoke(messages_3)

result_6 = result_5["messages"][-1].content
result_6

'quality assurance audits, records reviews and other activities under this Agreement through its (or its Affiliates’) employees or consultants with a bona fide need to know, but only to the extent necessary for the Party to exercise its rights and discharge its obligations under this Agreement,'